In [ ]:
import matplotlib.pyplot as plt
import os
import json
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
plt.rcParams['yaxis.labellocation'] = 'center'
plt.rcParams['xaxis.labellocation'] = 'center'
plt.rcParams['lines.markersize'] = 10
plt.rcParams['lines.markeredgewidth'] = 2.0
plt.rcParams['xtick.minor.top'] = False    # draw x axis top minor ticks
plt.rcParams['xtick.minor.bottom'] = False    # draw x axis bottom minor ticks
plt.rcParams['ytick.minor.left'] = True    # draw x axis top minor ticks
plt.rcParams['ytick.minor.right'] = True    # draw x axis bottom minor ticks
plt.rcParams['xtick.labelsize'] = 16
plt.rcParams['ytick.labelsize'] = 16
plt.rcParams['legend.fontsize'] = 16
plt.rcParams['font.size'] = 16

In [ ]:
total_num_samples = 455 # Replace with total number of training+evaluation samples

## Collect results

In [ ]:
base_dir = "results"  # Replace with your actual directory path
devices = ["cpu", "gpu"] # Replace with name of directories where results for each device are stored

# Initialize data storage
data = {device: {} for device in devices}

# Parse the directory structure
for device in devices:
    device_dir = os.path.join(base_dir, device)
    for bond_dir in os.listdir(device_dir):
        bond_path = os.path.join(device_dir, bond_dir)
        if not os.path.isdir(bond_path):
            continue
        
        bond_dim = int(bond_dir.split('_')[-1])
        train_times = []
        accuracies = []
        sensitivities = []
        specificities = []
        precisions = []
        F_measures = []
        throughputs = []

        # Read parameters.txt
        params_file = os.path.join(bond_path, "parameters.txt")
        loss_file = os.path.join(bond_path, "loss.npy")
        if os.path.exists(params_file) and os.path.exists(loss_file):
            with open(params_file, "r") as f:
                params = json.loads(f.read().split("Parameters: ")[-1])
                train_time_total = float(params["train_time"])
                accuracy = float(params["acc"])
                sensitivity = float(params["sensitivity"])
                specificity = float(params["specificity"])
                precision = float(params["precision"])
                F_measure = float(params["F_measure"])
                
            # Load the loss file to determine the number of epochs
            loss = np.load(loss_file)
            num_epochs = len(loss)
            train_time_per_epoch = train_time_total / num_epochs
            throughput = total_num_samples / train_time_per_epoch
            
            train_times.append(train_time_per_epoch)
            throughputs.append(throughput)
            accuracies.append(accuracy)
            sensitivities.append(sensitivity)
            specificities.append(specificity)
            precisions.append(precision)
            F_measures.append(F_measure)

        # Initialization of dictionary
        if bond_dim not in data[device]:
            data[device][bond_dim] = {"train_time": [], "acc": [], "throughput": [], "sensitivity": [], "specificity": [], "precision": [], "F_measure": []}
        
        data[device][bond_dim]["train_time"] = train_times
        data[device][bond_dim]["throughput"] = throughputs
        data[device][bond_dim]["acc"] = accuracies
        data[device][bond_dim]["sensitivity"] = sensitivities
        data[device][bond_dim]["specificity"] = specificities
        data[device][bond_dim]["precision"] = precisions
        data[device][bond_dim]["F_measure"] = F_measures
        

## Plot Throughput and Accuracy

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(8, 10))

colors = {'cpu': '#67a9cf', 'gpu': '#016c59'}

# First subplot: Throughput
for metric, ylabel in [("throughput", "Throughput (samples/s)")]:
    for device in devices:
        bond_dims = sorted(data[device].keys())
        metrics = [data[device][bond_dim][metric] for bond_dim in bond_dims]
        data[device][bond_dim][metric]
        axes[0].plot(bond_dims, metrics, label=f"{device.upper()}", marker="o", color=colors[device])
axes[0].set_xscale("log")
axes[0].set_ylabel("Throughput (samples/s)", fontsize=14, labelpad=15)
axes[0].grid(True, which="major", linestyle="--", linewidth=0.5)
axes[0].tick_params(axis='y', labelsize=15)
axes[0].tick_params(axis='x', labelsize=15)
axes[0].set_xticks(bond_dims)
axes[0].set_xticklabels(bond_dims)
axes[0].legend()

# Third subplot: Accuracy
for device in devices:
    bond_dims = sorted(data[device].keys())
    accuracies = [data[device][bond_dim]["acc"] for bond_dim in bond_dims]
    axes[1].plot(bond_dims, accuracies, label=device.upper(), marker="o", color=colors[device])
axes[1].set_xscale("log")
axes[1].set_xlabel("Bond Dimension", fontsize=14, labelpad=15)
axes[1].set_ylabel("Accuracy", fontsize=14, labelpad=15)
axes[1].grid(True, which="major", linestyle="--", linewidth=0.5)
axes[1].set_xticks(bond_dims)
axes[1].set_xticklabels(bond_dims)

# Set font size for x-ticks and y-ticks of the last subplot
axes[1].tick_params(axis='x', labelsize=15)
axes[1].tick_params(axis='y', labelsize=15)

labels = ['(a)', '(b)']
for ax, label in zip(axes, labels):
    ax.text(-0.1, 1.05, label, transform=ax.transAxes, fontsize=12, va='top', ha='right')

plt.tight_layout()
plt.savefig("results/plot.pdf")


## Plot sensitivity, specificity, precision, and F-measure

In [ ]:
for metric, ylabel in [("sensitivity", "Sensitivity"), ("specificity", "Specificity"), ("precision", "Precision"), ("F_measure", "F-measure")]:
    plt.figure(figsize=(10, 6))
    colors = {'cpu': 'cornflowerblue', 'gpu': 'forestgreen'}
    for device in devices:
        bond_dims = sorted(data[device].keys())
        metrics = [data[device][bond_dim][metric] for bond_dim in bond_dims]
        plt.plot(bond_dims, metrics, label=device.upper(), marker="o", color=colors[device])
    
    plt.xlabel("Bond Dimension")
    plt.ylabel(ylabel)
    plt.title(f"{ylabel} vs. Bond Dimension")
    plt.legend()
    plt.grid(True, which="both", linestyle="--", linewidth=0.5)
    plt.tight_layout()
    plt.show()